<a href="https://colab.research.google.com/github/2getsandesh/Flood_mapping/blob/main/Flood_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
import geemap

In [3]:
ee.Authenticate()

True

In [4]:
ee.Initialize(project='ee-xbosonai')

In [5]:
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

NASADEM: NASA NASADEM Digital Elevation 30m


In [6]:
import datetime

In [7]:
# Import required datasets
gsw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
hydrosheds = ee.Image("WWF/HydroSHEDS/03VFDEM")

In [8]:
beforeStart = '2018-07-15'
beforeEnd = '2018-08-10'
afterStart = '2018-08-10'
afterEnd = '2018-08-23'


beforeStart = ee.Date(beforeStart)
beforeEnd = ee.Date(beforeEnd)
afterStart = ee.Date(afterStart)
afterEnd = ee.Date(afterEnd)

In [9]:
# Load the GAUL feature collection
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Filter the feature collection to get the district of Ernakulam
ernakulam = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Ernakulam'))

# Get the geometry of the district
geometry = ernakulam.geometry()

# Create a Map object
Map = geemap.Map(center=[10.0, 76.5], zoom=8)

# Add the geometry to the map
Map.addLayer(geometry, {'color': 'grey'}, 'Ernakulam District')



In [10]:
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
              .filter(ee.Filter.eq('resolution_meters', 10))
              .filterBounds(geometry)
              .select(['VH']))


In [15]:
# Filter the collection based on before and after dates
beforeCollection = collection.filterDate(beforeStart, beforeEnd)
afterCollection = collection.filterDate(afterStart, afterEnd)

# Mosaic and clip images for before and after collections
before = beforeCollection.mosaic().clip(geometry)
after = afterCollection.mosaic().clip(geometry)


# Function to convert from dB
def toNatural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

# Function to convert to dB
def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

# Function to apply Refined Lee Speckle filter
def RefinedLee(img):
    # img must be in natural units, i.e., not in dB!
    # Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1, 3), 3)
    kernel3 = ee.Kernel.fixed(3, 3, weights3, 1, 1, False)

    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])
    sample_kernel = ee.Kernel.fixed(7, 7, sample_weights, 3, 3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # Duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "Collapse" the stack into a single band image (due to masking, each pixel has just one value (1-8) in its directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())

    # Calculate sample statistics
    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0, 0, 5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0, 7), 3).cat(ee.List.repeat(ee.List.repeat(1, 7), 4))
    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # Add the bands for rotated kernels
    for i in range(1, 4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))

    # "Collapse" the stack into a single band image (due to masking, each pixel has just one value in its directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # Generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    return result.arrayFlatten([['sum']])


# Apply speckle filtering to before and after images
beforeFiltered = toDB(RefinedLee(toNatural(before)))
afterFiltered = toDB(RefinedLee(toNatural(after)))

# Display the filtered images on the map
Map.addLayer(beforeFiltered, {'min': -25, 'max': 0}, 'Before Floods (Filtered)', True)
Map.addLayer(afterFiltered, {'min': -25, 'max': 0}, 'After Floods (Filtered)', True)

# Calculate the difference using division
difference = afterFiltered.divide(beforeFiltered)

# Define a threshold
diffThreshold = 1.25

# Initial estimate of flooded pixels
flooded = difference.gt(diffThreshold).rename('water').selfMask()

# Add the layer to the map
Map.addLayer(flooded, {'min': 0, 'max': 1, 'palette': ['orange']}, 'Initial Flood Area (Division)', True)

# Alternate way to detect change using subtraction
# Since the S1_GRD collection has log-scaled DB values, subtract is more appropriate
# If you use subtract(), the threshold value also changes.
difference = afterFiltered.subtract(beforeFiltered)
diffThreshold = -3

# Initial estimate of flooded pixels using subtraction
flooded = difference.lt(diffThreshold).rename('water').selfMask()

# Add the layer to the map
Map.addLayer(flooded, {'min': 0, 'max': 1, 'palette': ['purple']}, 'Initial Flood Area (subtract)', True)






In [16]:
# Mask out area with permanent/semi-permanent water
permanent_water = gsw.select('seasonality').gte(5).clip(geometry)
flooded = flooded.where(permanent_water, 0).selfMask()
Map.addLayer(permanent_water.selfMask(), {'min': 0, 'max': 1, 'palette': ['blue']}, 'Permanent Water')

# Mask out areas with more than 5 percent slope using the HydroSHEDS DEM
slope_threshold = 5
terrain = ee.Algorithms.Terrain(hydrosheds)
slope = terrain.select('slope').clip(geometry)
flooded = flooded.updateMask(slope.lt(slope_threshold))
Map.addLayer(slope.gte(slope_threshold).selfMask(), {'min': 0, 'max': 1, 'palette': ['cyan']}, 'Steep Areas', True)

# Remove isolated pixels
# connectedPixelCount is Zoom dependent, so visual result will vary
connected_pixel_threshold = 8
connections = flooded.connectedPixelCount(25)
flooded = flooded.updateMask(connections.gt(connected_pixel_threshold))
Map.addLayer(connections.lte(connected_pixel_threshold).selfMask(), {'min': 0, 'max': 1, 'palette': ['yellow']}, 'Disconnected Areas', True)

Map.addLayer(flooded, {'min': 0, 'max': 1, 'palette': ['red']}, 'Flooded Areas')

# Display the map
Map.addLayerControl()
Map



Map(bottom=495112.0, center=[10.067572275043405, 76.38039337385835], controls=(WidgetControl(options=['positio…

In [ ]:
'''# Calculate Affected Area
print('Total District Area (Ha)', geometry.area().divide(10000))

# Calculate the flooded area
stats = flooded.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=100,  # Adjusted scale for faster computation
    maxPixels=1e9,  # Adjusted maxPixels based on the size of the region
    tileScale=4
)
print('Flooded Area (Ha)', ee.Number(stats.get('water')).divide(10000))

# If the above computation times out, you can export it
flooded_area = ee.Number(stats.get('water')).divide(10000)
feature = ee.Feature(None, {'flooded_area': flooded_area})
fc = ee.FeatureCollection([feature])

# Export the flooded area data to Drive
task = ee.batch.Export.table.toDrive(
    collection=fc,
    description='Flooded_Area_Export',
    folder='earthengine',
    fileNamePrefix='flooded_area',
    fileFormat='CSV'
)
task.start()

'''


Total District Area (Ha) ee.Number({
  "functionInvocationValue": {
    "functionName": "Number.divide",
    "arguments": {
      "left": {
        "functionInvocationValue": {
          "functionName": "Geometry.area",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "Collection.geometry",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.loadTable",
                            "arguments": {
                              "tableId": {
                                "constantValue": "FAO/GAUL_SIMPLIFIED_500m/2015/level2"
                              }
                            }
                      